In [ ]:
import dask
from helpers.loading import *
from helpers import config


In [ ]:
all_data_bbo = {}
for market in config["transatlantic"]["markets"]["list"]:
    all_data_bbo[market] = load_all_data_dask(market, signal="bbo", precision="D",cols=["date","price","bid-price","ask-price"])

all_data_trade = {}
for market in config["transatlantic"]["markets"]["list"]:
    all_data_trade[market] = load_all_data_dask(market,precision="ms")

In [ ]:
all_trades = dd.concat(list(all_data_trade.values()))
all_trades.head()

### Daily number of trades

In [ ]:
daily_trades = all_trades.copy()
daily_trades["date"] = daily_trades["date"].round("D")
n_daily_trade_avg = daily_trades.groupby("date").count().mean().compute()

In [ ]:
print(f" there was {n_daily_trade_avg.price:0.1f} trades per day on average")

### Average period between trades 

In [ ]:
all_periods = all_trades.reset_index().date.diff(1).dropna()
all_periods < all_periods[pd.Timedelta(1, "h")]
mean_period = all_periods.mean().compute()

In [ ]:
print(f"there are {mean_period.total_seconds():0.2f}s on average between two trades")

### Average bid-ask spread 

In [ ]:
bbo_df = dd.concat(list(all_data_bbo.values()))
bbo_df.head()

In [ ]:
bbo_df["spread"] = bbo_df["bid-price"]-bbo_df["ask-price"]

In [ ]:
bbo_df["spread_ratio"] =  bbo_df["spread"].abs()/bbo_df["price"]*100
mean_spread = bbo_df.spread_ratio.mean().compute()

In [ ]:
print(f"The bid ask spread represents {mean_spread:0.2f}% of the mid price, in average")